In [2]:
import numpy as np
import pandas as pd
import src.miax_util as miax_util

In [3]:
monthly_closes = pd.read_parquet("resources/monthly_closes.parquet")
eligible_universe = pd.read_parquet("resources/eligible_universe.parquet")
rebalancing_dates = pd.read_parquet("resources/rebalancing_dates.parquet")

print(f"monthly_closes: {monthly_closes.shape}")
print(f"eligible_universe: {eligible_universe.shape}")
print(f"rebalancing_dates: {rebalancing_dates.shape}")

monthly_closes: (103686, 4)
eligible_universe: (64510, 2)
rebalancing_dates: (133, 3)


In [4]:


# Test con la primera fecha de rebalanceo
test_date = rebalancing_dates['date'].iloc[0]
test_eligible = eligible_universe[
    eligible_universe['rebal_date'] == test_date
]['symbol'].tolist()

test_result = miax_util.calculate_momentum(test_date, monthly_closes, test_eligible)
print(f"Fecha: {test_date}")
print(f"Activos con R_6 y R_12: {len(test_result)}")
print(test_result.head())

Fecha: 2015-01-30 00:00:00
Activos con R_6 y R_12: 487
                 R_12       R_6
symbol                         
A            0.008106  0.010069
AABA-201910  0.222352  0.376874
AAPL         0.340910  0.208658
ABBV         0.246221  0.201046
ABT          0.183019  0.128625


In [5]:

# Test con la primera fecha
test_scores = miax_util.calculate_scores(test_result)
print(f"Top 20 para {test_date}:")
print(test_scores[['R_12', 'R_6', 'Z_12', 'Z_6', 'score']])

Top 20 para 2015-01-30 00:00:00:
                 R_12       R_6      Z_12       Z_6     score
symbol                                                       
LUV          0.816718  0.475648  3.498600  2.277408  2.888004
EW           0.661163  0.450260  2.721030  2.136125  2.428577
EA           0.717584  0.291411  3.003063  1.252168  2.127616
VRTX         0.469330  0.497171  1.762118  2.397175  2.079647
MNST         0.469247  0.445768  1.761702  2.111131  1.936416
AGN-201503   0.650388  0.239186  2.667166  0.961550  1.814358
BBWI         0.376178  0.426531  1.296479  2.004079  1.650279
KR           0.499575  0.302867  1.913303  1.315917  1.614610
DAL          0.590748  0.213680  2.369045  0.819609  1.594327
COV-201501   0.427040  0.346874  1.550723  1.560809  1.555766
KMX          0.347831  0.407194  1.154781  1.896472  1.525627
LOW          0.345030  0.388568  1.140780  1.792823  1.466802
SIAL-201511  0.387252  0.335491  1.351835  1.497464  1.424649
CFN-201503   0.398914  0.323713  1.41

In [6]:
# NOTA: No filtramos el universo elegible por disponibilidad de precio en la fecha
# de rebalanceo. Hacerlo implicaría usar información futura (saber qué activos
# tendrán precio disponible ese día), lo que constituiría un lookahead bias.
# Los activos seleccionados sin precio en el día de rebalanceo serán tratados
# en el Notebook 4 como liquidez hasta el siguiente rebalanceo.

results = []

for _, row in rebalancing_dates.iterrows():
    rebal_date = row['date']

    # Activos elegibles para esta fecha
    eligible_symbols = eligible_universe[
        eligible_universe['rebal_date'] == rebal_date
    ]['symbol'].tolist()

    # Paso A: retornos acumulados
    momentum = miax_util.calculate_momentum(rebal_date, monthly_closes, eligible_symbols)

    # Pasos B y C: z-scores, score y top 20
    top20 = miax_util.calculate_scores(momentum)
    top20 = top20.reset_index()
    top20['rebal_date'] = rebal_date
    top20['weight'] = 0.05  # 5% cada activo

    results.append(top20)

# Concatenamos todos los resultados
portfolio = pd.concat(results, ignore_index=True)

print(f"Shape: {portfolio.shape}")
print(f"Fechas de rebalanceo: {portfolio['rebal_date'].nunique()}")
portfolio.head()

Shape: (2660, 8)
Fechas de rebalanceo: 133


,symbol,R_12,R_6,Z_12,Z_6,score,rebal_date,weight
0,LUV,0.816718,0.475648,3.498600,2.277408,2.888004,2015-01-30,0.05
1,EW,0.661163,0.450260,2.721030,2.136125,2.428577,2015-01-30,0.05
2,EA,0.717584,0.291411,3.003063,1.252168,2.127616,2015-01-30,0.05
3,VRTX,0.469330,0.497171,1.762118,2.397175,2.079647,2015-01-30,0.05
4,MNST,0.469247,0.445768,1.761702,2.111131,1.936416,2015-01-30,0.05


In [7]:
# ------------------------------------------------------------
# 5. GUARDAMOS EL CSV DE SALIDA
# ------------------------------------------------------------

portfolio.to_csv("resources/portfolio_selections.csv", index=False)
print("✅ portfolio_selections.csv guardado")
print(f"Total rebalanceos: {portfolio['rebal_date'].nunique()}")
print(f"Total filas: {len(portfolio)}")

✅ portfolio_selections.csv guardado
Total rebalanceos: 133
Total filas: 2660
